# Día 2: Context Managers

## Descripción General

Los context managers son una de las características más elegantes de Python para gestionar recursos. Permiten garantizar que los recursos (archivos, conexiones de red, locks, etc.) se adquieran y liberen correctamente, incluso cuando ocurren errores.

En este notebook aprenderás a usar el statement `with`, a crear tus propios context managers usando `__enter__` y `__exit__`, y a aprovechar el módulo `contextlib` para simplificar su implementación.

## Objetivos de Aprendizaje

Al finalizar este notebook, serás capaz de:

1. Usar el statement `with` para gestionar recursos de manera segura y automática
2. Entender el protocolo de context managers (`__enter__` y `__exit__`)
3. Crear context managers personalizados usando clases
4. Implementar context managers usando el decorador `@contextmanager` de contextlib
5. Identificar cuándo usar context managers para mejorar la robustez del código

## 1. El Statement `with`

### El Problema que Resuelve

Cuando trabajamos con recursos externos (archivos, conexiones de red, locks), debemos asegurarnos de liberarlos correctamente, incluso si ocurre un error. El enfoque tradicional con try/finally es verboso y propenso a errores.

In [ ]:
# BAD: Manual resource management - verbose and error-prone
file = open('example.txt', 'w')
try:
    file.write('Hello, World!')
    # If an error occurs here, the file might not be closed
finally:
    file.close()

In [ ]:
# GOOD: Using with statement - clean and safe
with open('example.txt', 'w') as file:
    file.write('Hello, World!')
    # File is automatically closed when exiting the block

### Aprendizaje Clave

El statement `with` garantiza que los recursos se liberen correctamente, incluso si ocurre una excepción. Es la forma pythónica de gestionar recursos y prevenir fugas de memoria o recursos bloqueados.

**Referencia oficial:** [PEP 343 - The with Statement](https://www.python.org/dev/peps/pep-0343/)

### Múltiples Context Managers

Podemos usar múltiples context managers en un solo statement:

In [ ]:
# Multiple context managers in one statement
with open('input.txt', 'r') as infile, open('output.txt', 'w') as outfile:
    content = infile.read()
    outfile.write(content.upper())
    # Both files are automatically closed

### Pregunta de Comprensión

¿Qué ventaja tiene usar `with` comparado con try/finally para gestionar archivos?

## 2. El Protocolo de Context Managers

### Cómo Funciona Internamente

Un context manager es cualquier objeto que implementa dos métodos especiales: `__enter__` y `__exit__`.

### Visualización del Flujo

```
with context_manager as variable:
    # code block

Flujo de ejecución:
1. Se llama a context_manager.__enter__()
2. El valor retornado se asigna a 'variable'
3. Se ejecuta el bloque de código
4. Se llama a context_manager.__exit__(exc_type, exc_val, exc_tb)
   - Siempre se ejecuta, incluso si hay excepciones
```

In [ ]:
# Example: Simple context manager class
class SimpleContextManager:
    """
    A basic context manager that demonstrates the protocol.
    
    This context manager prints messages when entering and exiting
    the context, helping visualize the execution flow.
    """
    
    def __enter__(self):
        """
        Called when entering the with block.
        
        :return: The resource to be used in the with block
        :rtype: str
        """
        print("Entering the context")
        return "Resource"
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """
        Called when exiting the with block.
        
        :param exc_type: Exception type if an exception occurred
        :type exc_type: type or None
        :param exc_val: Exception value if an exception occurred
        :type exc_val: Exception or None
        :param exc_tb: Exception traceback if an exception occurred
        :type exc_tb: traceback or None
        :return: True to suppress exception, False to propagate it
        :rtype: bool
        """
        print(f"Exiting the context (exception: {exc_type})")
        return False  # Don't suppress exceptions

# Using the context manager
with SimpleContextManager() as resource:
    print(f"Inside the context, using: {resource}")

### Aprendizaje Clave

El método `__enter__` se ejecuta al entrar al bloque `with` y retorna el recurso. El método `__exit__` se ejecuta al salir (siempre, incluso con excepciones) y recibe información sobre cualquier excepción que haya ocurrido.

**Referencia oficial:** [Context Manager Types - Python Documentation](https://docs.python.org/3/library/stdtypes.html#context-manager-types)

## 3. Context Managers Personalizados con Clases

### Caso de Uso: Timer

Creemos un context manager que mida el tiempo de ejecución de un bloque de código:

In [ ]:
import time

class Timer:
    """
    Context manager to measure execution time of a code block.
    
    Example:
        >>> with Timer("My operation"):
        ...     time.sleep(1)
        My operation took 1.00 seconds
    """
    
    def __init__(self, name: str = "Operation"):
        """
        Initialize the timer.
        
        :param name: Name of the operation being timed
        :type name: str
        """
        self.name = name
        self.start_time = None
    
    def __enter__(self):
        """
        Start the timer.
        
        :return: Self for optional use in with statement
        :rtype: Timer
        """
        self.start_time = time.time()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """
        Stop the timer and print elapsed time.
        
        :param exc_type: Exception type if an exception occurred
        :type exc_type: type or None
        :param exc_val: Exception value if an exception occurred
        :type exc_val: Exception or None
        :param exc_tb: Exception traceback if an exception occurred
        :type exc_tb: traceback or None
        :return: False to propagate exceptions
        :rtype: bool
        """
        elapsed = time.time() - self.start_time
        print(f"{self.name} took {elapsed:.2f} seconds")
        return False

# Using the Timer context manager
with Timer("Data processing"):
    # Simulate some work
    total = sum(range(1000000))
    print(f"Sum: {total}")

### Caso de Uso: Database Connection

Un patrón común es gestionar conexiones de base de datos:

In [ ]:
class DatabaseConnection:
    """
    Context manager for database connections.
    
    Ensures the connection is properly closed even if errors occur.
    """
    
    def __init__(self, connection_string: str):
        """
        Initialize with connection string.
        
        :param connection_string: Database connection string
        :type connection_string: str
        """
        self.connection_string = connection_string
        self.connection = None
    
    def __enter__(self):
        """
        Open the database connection.
        
        :return: The database connection object
        :rtype: object
        """
        print(f"Opening connection to {self.connection_string}")
        # In real code, this would create an actual connection
        self.connection = {"status": "connected"}
        return self.connection
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """
        Close the database connection.
        
        :param exc_type: Exception type if an exception occurred
        :type exc_type: type or None
        :param exc_val: Exception value if an exception occurred
        :type exc_val: Exception or None
        :param exc_tb: Exception traceback if an exception occurred
        :type exc_tb: traceback or None
        :return: False to propagate exceptions
        :rtype: bool
        """
        print("Closing database connection")
        if self.connection:
            self.connection["status"] = "closed"
        return False

# Using the DatabaseConnection context manager
with DatabaseConnection("postgresql://localhost/mydb") as conn:
    print(f"Connection status: {conn['status']}")
    # Perform database operations
# Connection is automatically closed here

### Aprendizaje Clave

Los context managers personalizados son ideales para encapsular lógica de setup/teardown. Cualquier recurso que requiera inicialización y limpieza es un buen candidato para un context manager.

**Referencia oficial:** [Context Manager Protocol - Python Data Model](https://docs.python.org/3/reference/datamodel.html#context-managers)

### Pregunta de Comprensión

¿Qué debe retornar el método `__exit__` si queremos que las excepciones se propaguen normalmente?